In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.getOrCreate()

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Define the schema
schema = StructType([
    StructField("sentiment", StringType(), True),
    StructField("publication_URL", StringType(), True),
    StructField("product_URL", StringType(), True),
    StructField("got_click", IntegerType(), True),
    StructField("gender", StringType(), True),
    StructField("age_group", StringType(), True)
])

log = spark.read.csv("/FileStore/tables/log.csv", schema=schema, header=False)
product = spark.read.csv("/FileStore/tables/products.csv", header=True)
product_cat = spark.read.csv("/FileStore/tables/product_categories.csv", header=True)

In [0]:
log.show()

+---------+--------------------+--------------------+---------+----------+----------+
|sentiment|     publication_URL|         product_URL|got_click|    gender| age_group|
+---------+--------------------+--------------------+---------+----------+----------+
| positive|https://www.foxne...|https://lees.com/...|        0|    female|  juvenile|
|  neutral|https://www.mirro...|https://coach.com...|        0|      male|     young|
| negative|https://www.nbcne...|https://covergirl...|        0|      male|middle-age|
| positive|https://www.exami...|https://covergirl...|        0|non-binary|  juvenile|
| negative|  https://www.nj.com|https://dell.com/...|        1|    female|     young|
| positive|https://www.chica...|https://remington...|        1|      male|    senior|
| negative|https://www.msn.c...|https://dell.com/...|        0|      male|     young|
| positive|https://www.salon...|https://samsung.c...|        1|      male|  juvenile|
| positive|https://www.cnet....|https://Ikea.com/...| 

In [0]:
product.show()

from pyspark.sql.functions import trim, lower

product = product.withColumn("product_type", trim(col("product_type")))
product = product.withColumn("product_URL", trim(col("product_URL")))
product.show(truncate=False)


+--------------------+--------------------+------------------+
|             product|         product_URL|      product_type|
+--------------------+--------------------+------------------+
|     Vitamix blender| https://vitamix....|           blender|
|       Lenova laptop| https://lenova.c...|          computer|
|InstantPot pressu...|https://InstantPo...|   pressure cooker|
|       NemoK blender|http://nemoK.co/b...|           blender|
|Hamilton Beach bl...|https://HamiltonB...|           blender|
|      Lavazza Coffee|https://Lavazza.c...|            coffee|
|    Starbucks Coffee|https://Starbucks...|            coffee|
|Centrum MultiVita...|https://centrum.c...|           vitamin|
|NordicTrack tread...|https://NordicTra...|         treadmill|
|   NordicTrack rower|https://NordicTra...|    rowing machine|
|Clinique moisturizer|https://clinique....|        face cream|
|NordicTrack ellip...|https://NordicTra...|elliptical trainer|
|               Tesla|   https://tesla.com|            

In [0]:
product_cat.show()
product_cat = product_cat.withColumn("category", trim(col("category")))
product_cat = product_cat.withColumn("product", trim(col("product")))

+------------------+--------------------+
|           product|            category|
+------------------+--------------------+
|           blender| small kitchen ap...|
|   pressure cooker| small kitchen ap...|
|          computer| consumer electro...|
|            coffee|       packaged food|
|           vitamin|              health|
|         treadmill|   fitness equipment|
|        face cream|     beauty products|
|    rowing machine|   fitness equipment|
|elliptical trainer|   fitness equipment|
|          lipstick|     beauty products|
|     women's purse|         accessories|
|             shoes|            footwear|
|               car|      transportation|
|             jeans|             apparel|
|        television|consumer electronics|
|            tablet|consumer electronics|
|         furniture|  household durables|
|            washer|large kitchen app...|
|             dryer|large kitchen app...|
|      refrigerator|large kitchen app...|
+------------------+--------------

For each product, compute all the Publication_URLs containing an ad for that product.

In [0]:
from pyspark.sql.functions import col, collect_list

joindf_1 = log.join(product, on = log.product_URL == product.product_URL, how = "right")
result_1 = joindf_1.groupBy("product").agg(collect_list("publication_URL").alias("Publication_URLs"))
result_1.show()

+--------------------+--------------------+
|             product|    Publication_URLs|
+--------------------+--------------------+
|       Lenova laptop|[https://www.tele...|
|        Docker pants|[https://www.bost...|
|         Coach purse|[https://www.al.c...|
|    Remington shaver|[https://www.vox....|
|       Maytag washer|[https://www.chic...|
|           Ikea sofa|[https://www.cbsn...|
|         Levis Jeans|[https://www.theg...|
|          Samsung TV|[https://www.theg...|
|  covergirl lipstick|[https://www.buzz...|
|    Covergirl makeup|[https://www.thea...|
|  Haier refrigerator|[https://www.foxn...|
|Centrum MultiVita...|[https://mashable...|
|Hamilton Beach bl...|                  []|
|          Apple iPad|[https://www.tele...|
|  Soundwave speakers|[https://www.nyda...|
|           LG washer|[https://www.npr....|
|      Samsung washer|[https://www.dail...|
|         Ford  sedan|[https://www.foxn...|
| Maytag refrigerator|[https://www.huff...|
|       Samsung dryer|[https://w

For each product type, compute all the Publication_URLs containing an ad for that product type.

In [0]:
result_2 = joindf_1.groupBy("product_type").agg(collect_list(joindf_1.publication_URL))
result_2.show()

+------------------+-----------------------------+
|      product_type|collect_list(publication_URL)|
+------------------+-----------------------------+
|        television|         [https://www.foxn...|
|      refrigerator|         [https://www.huff...|
|          lipstick|         [https://www.enga...|
|        face cream|         [https://www.usne...|
|           vitamin|         [https://mashable...|
|            tablet|         [https://www.tele...|
|    rowing machine|         [https://abcnews....|
|   pressure cooker|         [https://www.theg...|
|            shaver|         [https://www.vox....|
|            washer|         [https://www.chic...|
|            makeup|         [https://www.thea...|
|     women's purse|         [https://www.al.c...|
|             jeans|         [https://www.theg...|
|         furniture|         [https://www.cbsn...|
|             dryer|         [https://www.nyda...|
|elliptical trainer|                           []|
|          computer|         [h

For each product, compute the click rate for it. Click rate is the number of times a display of an ad was clicked on (by any user) divided by the number of times it was displayed (to any user). Note the click rate is not specific to each user.
- For each product, compute the click rate for each sentiment type

In [0]:
from pyspark.sql.functions import sum, count, col

joindf_2 = log.join(product, on = log.product_URL == product.product_URL, how = 'right')
result_3 = joindf_2.groupBy("product").agg((sum(col("got_click"))/count(col("got_click"))).alias("click_rate")).orderBy("product")
result_3.show(truncate=False, n=result_3.count())

+--------------------------+-------------------+
|product                   |click_rate         |
+--------------------------+-------------------+
|Apple computer            |0.7920792079207921 |
|Apple iPad                |0.5037878787878788 |
|Apple laptop              |0.5645161290322581 |
|BasilBasel perfume        |0.6493506493506493 |
|Broyhill recliner         |0.5392156862745098 |
|Centrum MultiVitamins     |0.6265560165975104 |
|Clinique moisturizer      |0.8055555555555556 |
|Coach purse               |0.388646288209607  |
|Cougar jeans              |0.2600732600732601 |
|Covergirl makeup          |0.2524752475247525 |
|Dell computer             |0.6515837104072398 |
|Dell laptop               |0.3151862464183381 |
|Docker pants              |0.6838709677419355 |
|Ford  sedan               |0.13656387665198239|
|Gillette shaver           |0.7133757961783439 |
|Giorgio perfume           |0.7990654205607477 |
|Givenchy perfume          |0.4597156398104265 |
|Guess perfume      

In [0]:
result_4 = joindf_2.groupBy("product","sentiment").agg((sum(col("got_click"))/count(col("got_click"))).alias("click_rate")).orderBy("product")
result_4.show(truncate=False)

+---------------------+---------+-------------------+
|product              |sentiment|click_rate         |
+---------------------+---------+-------------------+
|Apple computer       |neutral  |0.9230769230769231 |
|Apple computer       |positive |0.7647058823529411 |
|Apple computer       |negative |0.6956521739130435 |
|Apple iPad           |negative |0.391304347826087  |
|Apple iPad           |neutral  |0.627906976744186  |
|Apple iPad           |positive |0.5                |
|Apple laptop         |neutral  |0.7317073170731707 |
|Apple laptop         |negative |0.15384615384615385|
|Apple laptop         |positive |0.7727272727272727 |
|BasilBasel perfume   |neutral  |0.864406779661017  |
|BasilBasel perfume   |positive |0.36666666666666664|
|BasilBasel perfume   |negative |0.7714285714285715 |
|Broyhill recliner    |neutral  |0.5625             |
|Broyhill recliner    |positive |0.24285714285714285|
|Broyhill recliner    |negative |0.8142857142857143 |
|Centrum MultiVitamins|neutr

For each product type, compute the click rate for it.
- For each product type, compute the click rate for each sentiment type

In [0]:
result_5 = joindf_2.groupBy("product_type").agg((sum(col("got_click"))/count(col("got_click"))).alias("click_rate")).orderBy("product_type")
result_5.show(n=result_5.count())

+------------------+-------------------+
|      product_type|         click_rate|
+------------------+-------------------+
|           blender| 0.5412026726057907|
|               car|0.37044967880085655|
|            coffee| 0.3552941176470588|
|          computer| 0.4996549344375431|
|             dryer|0.45271629778672035|
|elliptical trainer|               null|
|        face cream| 0.8055555555555556|
|         furniture| 0.5549738219895288|
|             jeans|0.45147679324894513|
|          lipstick| 0.6637426900584795|
|            makeup| 0.2524752475247525|
|             pants| 0.6838709677419355|
|           perfume| 0.5668934240362812|
|   pressure cooker|                0.5|
|      refrigerator| 0.2872727272727273|
|    rowing machine|0.22340425531914893|
|            shaver|               0.54|
|          speakers| 0.5359801488833746|
|            tablet| 0.5037878787878788|
|        television| 0.5320910973084886|
|         treadmill| 0.4897119341563786|
|           vita

In [0]:
result_6 = joindf_2.groupBy("product_type","sentiment").agg((sum(col("got_click"))/count(col("got_click"))).alias("click_rate")).orderBy("product_type")
result_6.show(n=result_6.count())

+------------------+---------+-------------------+
|      product_type|sentiment|         click_rate|
+------------------+---------+-------------------+
|           blender|  neutral|0.33112582781456956|
|           blender| positive|0.47580645161290325|
|           blender| negative| 0.7701149425287356|
|           blender|     null|               null|
|               car| positive| 0.1566265060240964|
|               car|  neutral| 0.5816993464052288|
|               car| negative| 0.3918918918918919|
|            coffee| negative| 0.4652777777777778|
|            coffee| positive|0.24113475177304963|
|            coffee|  neutral|0.35714285714285715|
|          computer|  neutral| 0.5161290322580645|
|          computer| positive| 0.5705128205128205|
|          computer| negative| 0.4144329896907217|
|             dryer| negative| 0.3236994219653179|
|             dryer|  neutral| 0.7345679012345679|
|             dryer| positive|0.30864197530864196|
|elliptical trainer|     null| 

For each category, compute the click rate for it.
- Similar to above, for each category compute the click rate for each sentiment type

In [0]:
joindf_3 = joindf_2.join(product_cat, on = joindf_2.product_type == product_cat.product, how = "right")
#joindf_3.show(truncate=False)
#joindf_2.show(truncate=False)

result_7 = joindf_3.groupBy("category").agg((sum(col("got_click"))/count(col("got_click"))).alias("click_rate")).orderBy("category")
result_7.show(truncate=False)

result_8 = joindf_3.groupBy("category","sentiment").agg((sum(col("got_click"))/count(col("got_click"))).alias("click_rate")).orderBy("category")
result_8.show(truncate=False)


+------------------------+-------------------+
|category                |click_rate         |
+------------------------+-------------------+
|accessories             |0.5151515151515151 |
|apparel                 |0.4930715935334873 |
|beauty products         |0.5797807551766139 |
|consumer electronics    |0.514660227664712  |
|fitness equipment       |0.37354988399071926|
|footwear                |null               |
|health                  |0.6265560165975104 |
|household durables      |0.5549738219895288 |
|large kitchen appliances|0.4564369310793238 |
|packaged food           |0.3552941176470588 |
|small kitchen appliances|0.5288611544461779 |
|transportation          |0.37044967880085655|
+------------------------+-------------------+

+--------------------+---------+-------------------+
|category            |sentiment|click_rate         |
+--------------------+---------+-------------------+
|accessories         |positive |0.45751633986928103|
|accessories         |neutral  |0.6

Choose a product randomly; determine if there are any 'significant' differences in the click rate between positive and negative sentiment type of the ad context for that product type given the gender of the viewer.
- You can use any form of statistical testing to calculate "significance" that you like. Eg. you assume a binomial distribution for click rate and use a z-test. Compute these figures for all 3 genders.

In [0]:
import random
from pyspark.sql.functions import col, sum, count

# Select a random product_type
random_product = joindf_2.select("product_type").distinct().collect()
selected_product = random.choice(random_product)["product_type"]

# Print the selected product
print(f"Randomly selected product_type: {selected_product}")

# Filter data for the selected product_type
filtered_data = joindf_2.filter(col("product_type") == selected_product)
filtered_data.show(truncate=False)

# Group by sentiment and gender to calculate clicks and views
click_rate_data = filtered_data.groupBy("sentiment", "gender").agg(
    sum(col("got_click")).alias("total_clicks"),
    count(col("got_click")).alias("total_views")
).withColumn("click_rate", col("total_clicks") / col("total_views"))

click_rate_data.show(truncate=False)

from scipy.stats import norm
import math

# Collect the data into a Pandas DataFrame
click_rate_df = click_rate_data.toPandas()

# Filter data for positive and negative sentiment
positive = click_rate_df[click_rate_df["sentiment"] == "positive"]
negative = click_rate_df[click_rate_df["sentiment"] == "negative"]

# Perform Z-test for each gender
print(f"Analysis for Product Type: {selected_product}\n")
for gender in ["male", "female", "non-binary"]:
    pos_data = positive[positive["gender"] == gender]
    neg_data = negative[negative["gender"] == gender]
    
    if len(pos_data) > 0 and len(neg_data) > 0:
        # Extract values
        p1 = pos_data["click_rate"].values[0]
        p2 = neg_data["click_rate"].values[0]
        n1 = pos_data["total_views"].values[0]
        n2 = neg_data["total_views"].values[0]
        
        # Combined click rate
        p_combined = (p1 * n1 + p2 * n2) / (n1 + n2)
        
        # Z-test formula
        z = (p1 - p2) / math.sqrt(p_combined * (1 - p_combined) * (1/n1 + 1/n2))
        p_value = 2 * (1 - norm.cdf(abs(z)))  # Two-tailed test
        
        print(f"Gender: {gender}")
        print(f"Z-statistic: {z:.4f}, P-value: {p_value:.4f}")
        print(f"Significant Difference: {'Yes' if p_value < 0.05 else 'No'}\n")
    else:
        print(f"Not enough data for gender: {gender}")


Randomly selected product_type: lipstick
+---------+-------------------------------+------------------------------+---------+------+----------+-------------------+------------------------------+------------+
|sentiment|publication_URL                |product_URL                   |got_click|gender|age_group |product            |product_URL                   |product_type|
+---------+-------------------------------+------------------------------+---------+------+----------+-------------------+------------------------------+------------+
|neutral  |https://www.engadget.com/      |http://maybelline.com/lipstick|0        |female|senior    |Maybelline lipstick|http://maybelline.com/lipstick|lipstick    |
|positive |https://www.vice.com/en_us     |http://maybelline.com/lipstick|1        |male  |young     |Maybelline lipstick|http://maybelline.com/lipstick|lipstick    |
|neutral  |https://www.dallasnews.com/    |http://maybelline.com/lipstick|0        |male  |middle-age|Maybelline lipstick|ht